In [ ]:
using CSV, DataFrames, Plots, Statistics, JuMP, Gurobi

In [ ]:
pollution = DataFrame(CSV.File("Data/pollution_location.csv"))
pollution_xy = pollution[:,1:2]|>Matrix
sensors_xy = DataFrame(CSV.File("Data/sensors_location.csv"))[:,1:2]|>Matrix

lat_to_km = [111, 92]
lat_diff = (pollution_xy[:,1].-sensors_xy[:,1]').*lat_to_km[1]
long_diff = (pollution_xy[:,2].-sensors_xy[:,2]').*lat_to_km[2]

dist_matrix = sqrt.(lat_diff.^2 + long_diff.^2)'
n, m = size(dist_matrix)

In [ ]:
q = pollution.AIQ
r = 2
p = sum(q)/n  # unit price

In [ ]:
# q = f(quality)

In [ ]:
d = dist_matrix.<=r;

In [ ]:
function model1(mu)
    model = Model(Gurobi.Optimizer);
    set_optimizer_attribute(model, "TimeLimit", 5);
    
    @variable(model, x[1:n], Bin) #Whether we put a censor at spot i
    @variable(model, y[1:m].<=1) #Whether we spot j is covered
    
    @constraint(model, y'.<= x'*d)
    # @constraint(model, [i=1:n],x[i]<=sum(d[i])) #censors that do not capture any points are deleted.
    
    
    @objective(model, Max, mu .* sum(y.*q) - (1-mu)*p*sum(x))
    optimize!(model)
    return value.(x), value.(y)
end
x, y = model1(0.2)
sum(y)

In [ ]:
scatter(pollution_xy[:,2], pollution_xy[:,1], marker_z = y, color=cgrad([:lightgray, :green]), markersize = 1.5, 
shape = :rect,  markerstrokewidth = 0,ratio = 1.1,
legend=false, axis=false, grid=false)
title!("Solution (we can relax the y :') ! )")
index_x = findall(x.==1)
scatter!(sensors_xy[index_x,2], sensors_xy[index_x,1], color=:black, markersize = 2,shape = :circle,  markerstrokewidth = 0)